## TestSGF1

The goal of this notebook attempts to check the correctness of the Sentiment Graph F1 evaluation script.

In [25]:
import os, sys, json

In [26]:
# makedirs if not exist
os.makedirs("../../etl/data/processed/TestSGF1", exist_ok=True)

In [27]:
GOLD_PATH="../../etl/data/processed/TestSGF1/ds_gold.json"
MAN_PATH="../../etl/data/processed/TestSGF1/ds_man.json"

In [28]:
def save_dict_as_json(dict_data, filename):
    with open(filename, 'w') as f:
        json.dump(dict_data, f)

In [29]:
test_ds_gold = [{'sent_id': '0',
  'text': 'Oshkosh geniesst den Ausblick auf den Genfersee .',
  'opinions': [{'Source': [['Oshkosh'], ['0:7']],
    'Polar_expression': [['geniesst'], ['8:16']],
    'Target': [['Ausblick'], ['21:29']],
    'Polarity': 'Positive',
    'Intensity': 'Average'}]},
 {'sent_id': '1',
  'text': 'Die Opposition will den Unmut ausnutzen und am nächsten Samstag eine weitere Kundgebung durchführen .',
  'opinions': [{'Source': [['Opposition'], ['4:14']],
    'Target': [['Unmut'], ['24:29']],
    'Polar_expression': [['ausnutzen'], ['30:39']],
    'Polarity': 'Negative',
    'Intensity': 'Average'}]},
 {'sent_id': '2',
  'text': 'Deutsches Laisser-faire dagegen verträgt sich nicht mit der politischen und ökonomischen Realität .',
  'opinions': [{'Source': [['Laisser-faire'], ['10:23']],
    'Polar_expression': [['verträgt'], ['32:40']],
    'Target': [['Realität'], ['89:97']],
    'Polarity': 'Negative',
    'Intensity': 'Average'}]},
 {'sent_id': '3',
  'text': 'Ihr geistiger Vater Valentin Oehen tritt heute tatsächlich an Versammlungen der rechtsextremen Partei National Orientierter Schweizer auf ; gleichzeitig klagt er , die Grünliberalen hätten seine Ideen geklaut .',
  'opinions': [{'Source': [['Grünliberalen'], ['168:181']],
    'Target': [['Ideen'], ['195:200']],
    'Polar_expression': [['geklaut'], ['201:208']],
    'Polarity': 'Neutral',
    'Intensity': 'Average'}]},
]

In [30]:
import copy

def copy_list(ds_list):
    return copy.deepcopy(ds_list)

def save_two_files(ds_gold, ds_mod):
    save_dict_as_json(ds_gold, GOLD_PATH)
    save_dict_as_json(ds_mod, MAN_PATH)

### Test 1, alternate on the expressions

Something strange appears to happen when the spans do not align propery, resp. if they go beyond a word, then the score increases strangely.

In [112]:
# 
test_ds_t1 = copy_list(test_ds_gold)

# test1 -> s-f1: .75, t-f1: 1, e-f1: 1, st-measures: 75

# test_ds_t1[0]["opinions"][0]["Source"] = [['Ausblick'], ['21:29']]
#test_ds_t1[1]["opinions"][0]["Source"] = [['Unmut'], ['24:29']]
#test_ds_t1[2]["opinions"][0]["Source"] = [['Realität'], ['89:97']]
#test_ds_t1[3]["opinions"][0]["Source"] = [['Ideen'], ['195:200']]

# test2 
# unlabeled SF1 should be higher, than not labeled

# test_ds_t1[2]["opinions"][0]["Polarity"] = "Positive"

# test3
# remove an opinion
# test_ds_t1[1]["opinions"] = []

# test4
# remove and entire sentence -> should raise an error, not allowed!
# test_ds_t1.pop(0)

# test5
# different sentences -> tuple score decreases, but ste scores unaffected?!
# since it is only evaluated one-on-one (per-tuple)?!
test_ds_t1[1]["text"] = "abcd"

# test6
# swapped IDs
temp_id = test_ds_t1[1]["sent_id"]
test_ds_t1[1]["sent_id"] = test_ds_t1[2]["sent_id"]
test_ds_t1[2]["sent_id"] = temp_id


save_two_files(test_ds_gold, test_ds_t1)

In [113]:
# test_ds_t1

In [114]:

!(source ~/envs/perin-venv/bin/activate && cd ../../external_repos/direct_parsing_to_sent_graph/evaluation && python evaluate_single_dataset.py ../../../etl/data/processed/TestSGF1/ds_gold.json ../../../etl/data/processed/TestSGF1/ds_man.json)


Source F1: 1.000
Target F1: 1.000
Expression F1: 1.000
Unlabeled Sentiment Tuple F1: 0.500
Sentiment Tuple F1: 0.500
{
  "source/f1": 0.9999992500003125,
  "target/f1": 0.9999992500003125,
  "expression/f1": 0.9999992500003125,
  "sentiment_tuple/unlabeled_f1": 0.5,
  "sentiment_tuple/precision": 0.5,
  "sentiment_tuple/recall": 0.5,
  "sentiment_tuple/f1": 0.5
}

[0.9999992500003125, 0.9999992500003125, 0.9999992500003125, 0.5, 0.5, 0.5, 0.5]
